In [ ]:
global in_sample_dataset
global out_sample_dataset


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense, Flatten, Conv1D
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.multivariate.pca import PCA
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from math import sqrt
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
from statsmodels.tsa.api import VAR
import numpy as np
import seaborn as sns
sns.set(rc={'figure.figsize':(8.7,6.27)})

from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.layers import LeakyReLU

import os

In [ ]:
!unzip testmodel.zip

Archive:  testmodel.zip
replace saved_models/my_model/keras_metadata.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: saved_models/my_model/keras_metadata.pb  
replace saved_models/my_model/variables/variables.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: saved_models/my_model/variables/variables.data-00000-of-00001  
replace saved_models/my_model/variables/variables.index? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: saved_models/my_model/variables/variables.index  
replace saved_models/my_model/saved_model.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: saved_models/my_model/saved_model.pb  
replace saved_models/discrete/keras_metadata.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: saved_models/discrete/keras_metadata.pb  
replace saved_models/discrete/variables/variables.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: saved_models/discrete/variables/variables.data-00000-of-00001  
replace saved_

In [ ]:
new_model = tf.keras.models.load_model('saved_models/discrete')


In [ ]:
class UndefinedModelError(Exception):
    """Please input either 'discrete' or 'time-window' as model type"""
    def __init__(self, message="Please input either 'discrete' or 'time_window' as model type"):
        self.message = message
        super().__init__(self.message)
    pass

In [ ]:
def get_metrics(errors, actual, predicted, name, sample):
  errors = np.array(errors)
  mape = np.mean(np.abs(errors)/actual)
  accuracy = 100- (100*mape)
  rmse = np.sqrt(np.mean((np.square(errors))))
  rmse_percent = rmse/np.mean(actual)
  mae = np.mean(np.abs(errors))
  mae_percent = np.mean(np.abs(errors))/np.mean(actual)
  metrics = {"MAPE":[mape], "Accuracy%":[round(accuracy, 2)], 
             "RMSE": [round(accuracy, 2)], "RMSE%": [round(100*rmse,2)], 
             "MAE":[round(mae,2)], "MAE%":[round(100*mae,2)]}

  metrics_df = pd.DataFrame.from_dict(metrics)
  filename = sample+ "_"+ name+ "_performance_metrics"
  print(filename)
  metrics_df.to_csv(filename+".csv")
  return filename
  
def make_visualisations(errors, actual, predicted, filename):
  results = pd.DataFrame(columns=["Expected", "Predicted"])
  results["Expected"] = actual
  #results["Expected_Trend"] = results['Expected'].rolling(3).mean()

  results["Predicted"] = predicted
  #results["Predicted_Trend"] = results['Predicted'].rolling(3).mean()

  #plt.ylim(0,5)
  sns.set(rc={'figure.figsize':(12.7,6.27)})
  sns.lineplot(data=results,palette="deep").set_title(filename)
  plt.savefig(filename + ".png")




def show_plot(plot_data, delta, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.ylim([0, 5])
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()
    return

def make_predictions_in_sample(in_sample_dataset, model):
  errors = []
  actual = []
  predicted = []
  for x, y in in_sample_dataset.take(200):
    # show_plot(
    #     [x[0][:, 4].numpy(), y[0].numpy(), model.predict(x)[0]],
    #     1,
    #     "Single Step Prediction",
    # )

    errors.append(y[0].numpy()[0]- model.predict(x)[0][0])

    actual.append(y[0].numpy()[0])
    predicted.append(model.predict(x)[0][0])
  return errors, actual, predicted

def make_predictions_out_sample(model):
  errors = []
  actual = []
  predicted = []
  for x, y in out_sample_dataset.take(200):
    show_plot(
        [x[0][:, 4].numpy(), y[0].numpy(), model.predict(x)[0]],
        1,
        "Single Step Prediction",
    )

    errors.append(y[0].numpy()[0]- model.predict(x)[0][0])

    actual.append(y[0].numpy()[0])
    predicted.append(model.predict(x)[0][0])
  return errors, actual, predicted

def eval_cnn(type = None):
  if type =='discrete':
    past = 1
  elif type=='time_window':
    past = 3
  else:
    raise UndefinedModelError
  
  step = 1
  sequence_length = int(past / step)
  in_sample_data = pd.read_csv("in_sample.csv",index_col=0)
  in_sample_x = in_sample_data[["0","1","2","3","5"]].values
  in_sample_y = in_sample_data[["4"]].values
  out_sample_data = pd.read_csv("out_sample.csv",index_col=0)
  out_sample_x = in_sample_data[["0","1","2","3","5"]].values
  out_sample_y = in_sample_data[["4"]].values


  in_sample_dataset = keras.preprocessing.timeseries_dataset_from_array(
    in_sample_x,
    in_sample_y,
    sequence_length=sequence_length,
    sampling_rate=step,
    )
  
  
  model_name = "saved_models/classical_"+type
  model = tf.keras.models.load_model(model_name)
  errors, actual, predicted = make_predictions_in_sample(in_sample_dataset, model)
  name = "classical_"+type
  filename = get_metrics(errors, actual, predicted, name, "in_sample")
  

  out_sample_dataset = keras.preprocessing.timeseries_dataset_from_array(
    out_sample_x,
    out_sample_y,
    sequence_length=sequence_length,
    sampling_rate=step,
    )
  
  
  errors, actual, predicted = make_predictions_out_sample(model)
  filename = get_metrics(errors, actual, predicted, name, "out_of_sample")


eval_cnn("discrete")


in_sample_classical_discrete_performance_metrics


NameError: ignored

in_sample_saved_models/classical_discrete_performance_metrics


FileNotFoundError: ignored

In [ ]:
metrics = pd.DataFrame(columns=[])